In [1]:
import teehr
from pathlib import Path
import pandas as pd
import numpy as np
import shutil

from teehr import TimeseriesAwareCalculatedFields as tcf

In [2]:
"""  ## SMALL EVAL

# set up the eval directory, clear contents if any exist
test_eval_dir = Path(Path.home(), 'temp', 'metric_numpy_test')
if test_eval_dir.exists():
    shutil.rmtree(test_eval_dir)
    print("Removed the existing test eval directory...")
test_eval_dir.mkdir(parents=True, exist_ok=True)
print(f"Created the test eval directory at {test_eval_dir}")

# copy contents of s3 directory to the test directory folder
s3_e0_dir = Path(Path.home(), 'temp', 's3_e0_08122025')
shutil.copytree(s3_e0_dir, test_eval_dir, dirs_exist_ok=True)
print(f"Copied contents from {s3_e0_dir} to {test_eval_dir}") """

'  ## SMALL EVAL\n\n# set up the eval directory, clear contents if any exist\ntest_eval_dir = Path(Path.home(), \'temp\', \'metric_numpy_test\')\nif test_eval_dir.exists():\n    shutil.rmtree(test_eval_dir)\n    print("Removed the existing test eval directory...")\ntest_eval_dir.mkdir(parents=True, exist_ok=True)\nprint(f"Created the test eval directory at {test_eval_dir}")\n\n# copy contents of s3 directory to the test directory folder\ns3_e0_dir = Path(Path.home(), \'temp\', \'s3_e0_08122025\')\nshutil.copytree(s3_e0_dir, test_eval_dir, dirs_exist_ok=True)\nprint(f"Copied contents from {s3_e0_dir} to {test_eval_dir}") '

In [3]:
## BIGGER EVAL

# set up the eval directory, clear contents if any exist
test_eval_dir = Path(Path.home(), 'temp', 'metric_numpy_test')
if test_eval_dir.exists():
    shutil.rmtree(test_eval_dir)
    print("Removed the existing test eval directory...")
test_eval_dir.mkdir(parents=True, exist_ok=True)
print(f"Created the test eval directory at {test_eval_dir}")

# copy contents of s3 directory to the test directory folder
PSU_dir = Path(Path.home(), 'temp', 'PSU_data', 'PSU_evaluation')
shutil.copytree(PSU_dir, test_eval_dir, dirs_exist_ok=True)
print(f"Copied contents from {PSU_dir} to {test_eval_dir}")

Removed the existing test eval directory...
Created the test eval directory at /home/slandsteiner/temp/metric_numpy_test
Copied contents from /home/slandsteiner/temp/PSU_data/PSU_evaluation to /home/slandsteiner/temp/metric_numpy_test


In [4]:
ev = teehr.Evaluation(test_eval_dir)

Version file not found in /home/slandsteiner/temp/metric_numpy_test.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/02 15:27:36 WARN Utils: Your hostname, RTI-504155, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/02 15:27:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/slandsteiner/repos/teehr/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/slandsteiner/.ivy2.5.2/cache
The jars for the packages stored in: /home/slandsteiner/.ivy2.5.2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f4eabb1b-ba51-4270-96fe-0aa41b0b6288;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.4.1 in central
	found software.amazon.awssdk#bundle;2.24.6 in central
	found org.wildfly.ope

In [5]:
sdf = ev.joined_timeseries.to_sdf()
sdf.show()

+--------------+-------------------+-------------------+---------------------+-------------+---------------+---------+------+------------------+--------------------+
|reference_time|         value_time|primary_location_id|secondary_location_id|primary_value|secondary_value|unit_name|member|configuration_name|       variable_name|
+--------------+-------------------+-------------------+---------------------+-------------+---------------+---------+------+------------------+--------------------+
|          NULL|1993-03-24 00:00:00|      usgs-06799100|        usgs-06799100|     9.542777|      13.559723|    m^3/s|  None|          d_sacsma|streamflow_daily_...|
|          NULL|1992-10-26 00:00:00|      usgs-01011000|        usgs-01011000|     33.93774|       71.04006|    m^3/s|  None|          d_sacsma|streamflow_daily_...|
|          NULL|1993-04-07 00:00:00|      usgs-06799100|        usgs-06799100|     10.67545|      14.996458|    m^3/s|  None|          d_sacsma|streamflow_daily_...|
|   

In [6]:
# get minimum date and maximum date from value_time column
min_date = sdf.agg({"value_time": "min"}).collect()[0][0]
max_date = sdf.agg({"value_time": "max"}).collect()[0][0]
print(f"Data ranges from {min_date} to {max_date}")

# get length of the jts per primary_location_id
jts_length = sdf.count()
number_of_locations = sdf.select("primary_location_id").distinct().count()
print(f"Number of rows in joined timeseries: {jts_length}")
print(f"Number of distinct primary_location_id: {number_of_locations}")
print(f"Average number of rows per primary_location_id: {jts_length/number_of_locations}")

Data ranges from 1992-09-30 19:00:00 to 2010-09-28 19:00:00


Number of rows in joined timeseries: 58152288
Number of distinct primary_location_id: 2572
Average number of rows per primary_location_id: 22609.75427682737


In [7]:
# get list of first 30 unique primary location ids
unique_ids = sdf.select('primary_location_id').distinct().rdd.map(lambda row: row[0]).collect()
unique_ids = unique_ids[:30]
print(f"Unique primary location ids: {unique_ids}")

Unique primary location ids: ['usgs-01153550', 'usgs-07336200', 'usgs-07362587', 'usgs-01398500', 'usgs-08200000', 'usgs-01447680', 'usgs-09124500', 'usgs-01541000', 'usgs-09247600', 'usgs-11376000', 'usgs-02421000', 'usgs-02473500', 'usgs-03106000', 'usgs-07335300', 'usgs-01467150', 'usgs-10166430', 'usgs-10242000', 'usgs-11460750', 'usgs-03168000', 'usgs-04243500', 'usgs-01124000', 'usgs-09128000', 'usgs-01573000', 'usgs-10343500', 'usgs-11124500', 'usgs-12027500', 'usgs-12324200', 'usgs-12395000', 'usgs-02455980', 'usgs-02474560']


In [8]:
test = ev.metrics.query(
    filters=[
        "primary_location_id IN ({})".format(', '.join(f"'{id}'" for id in unique_ids))
    ],
    group_by=['primary_location_id','configuration_name'],
    order_by=['primary_location_id','configuration_name'],
    include_metrics=[
        teehr.DeterministicMetrics.RelativeBias(),
    ]
).to_sdf()

test.show()

+-------------------+------------------+-------------+
|primary_location_id|configuration_name|relative_bias|
+-------------------+------------------+-------------+
|      usgs-01124000|        d_ensemble|  0.028561877|
|      usgs-01124000|             d_hbv|   0.03245213|
|      usgs-01124000|            d_prms|  0.038089544|
|      usgs-01124000|          d_sacsma|  0.038089544|
|      usgs-01153550|        d_ensemble|   0.04093089|
|      usgs-01153550|             d_hbv|  0.012490803|
|      usgs-01153550|            d_prms|   0.05484145|
|      usgs-01153550|          d_sacsma|   0.05484145|
|      usgs-01398500|        d_ensemble|   0.03511029|
|      usgs-01398500|             d_hbv|  -0.02304061|
|      usgs-01398500|            d_prms| 0.0028719162|
|      usgs-01398500|          d_sacsma| 0.0028719166|
|      usgs-01447680|        d_ensemble|  -0.04990004|
|      usgs-01447680|             d_hbv| -0.047125384|
|      usgs-01447680|            d_prms| -0.014411413|
|      usg

In [9]:
test = ev.metrics.query(
    filters=[
        "primary_location_id IN ({})".format(', '.join(f"'{id}'" for id in unique_ids))
    ],
    group_by=['primary_location_id','configuration_name'],
    order_by=['primary_location_id','configuration_name'],
    include_metrics=[
        teehr.DeterministicMetrics.MeanAbsoluteRelativeError(),
    ]
).to_sdf()

test.show()

+-------------------+------------------+----------------------------+
|primary_location_id|configuration_name|mean_absolute_relative_error|
+-------------------+------------------+----------------------------+
|      usgs-01124000|        d_ensemble|                  0.22769767|
|      usgs-01124000|             d_hbv|                  0.26217392|
|      usgs-01124000|            d_prms|                  0.23940893|
|      usgs-01124000|          d_sacsma|                  0.23940893|
|      usgs-01153550|        d_ensemble|                   0.3041933|
|      usgs-01153550|             d_hbv|                  0.35183188|
|      usgs-01153550|            d_prms|                   0.3402173|
|      usgs-01153550|          d_sacsma|                   0.3402173|
|      usgs-01398500|        d_ensemble|                   0.3012323|
|      usgs-01398500|             d_hbv|                  0.31756875|
|      usgs-01398500|            d_prms|                  0.31320545|
|      usgs-01398500

In [10]:
test = ev.metrics.query(
    filters=[
        "primary_location_id IN ({})".format(', '.join(f"'{id}'" for id in unique_ids))
    ],
    group_by=['primary_location_id','configuration_name'],
    order_by=['primary_location_id','configuration_name'],
    include_metrics=[
        teehr.DeterministicMetrics.NashSutcliffeEfficiency(),
    ]
).to_sdf()

test.show()

+-------------------+------------------+-------------------------+
|primary_location_id|configuration_name|nash_sutcliffe_efficiency|
+-------------------+------------------+-------------------------+
|      usgs-01124000|        d_ensemble|                0.8676816|
|      usgs-01124000|             d_hbv|               0.82247573|
|      usgs-01124000|            d_prms|               0.84164375|
|      usgs-01124000|          d_sacsma|               0.84164375|
|      usgs-01153550|        d_ensemble|               0.76861763|
|      usgs-01153550|             d_hbv|               0.65908766|
|      usgs-01153550|            d_prms|               0.74809736|
|      usgs-01153550|          d_sacsma|                0.7480974|
|      usgs-01398500|        d_ensemble|               0.56605387|
|      usgs-01398500|             d_hbv|                0.5427208|
|      usgs-01398500|            d_prms|               0.54573154|
|      usgs-01398500|          d_sacsma|               0.54573

In [11]:
test = ev.metrics.query(
    filters=[
        "primary_location_id IN ({})".format(', '.join(f"'{id}'" for id in unique_ids))
    ],
    group_by=['primary_location_id','configuration_name'],
    order_by=['primary_location_id','configuration_name'],
    include_metrics=[
        teehr.DeterministicMetrics.KlingGuptaEfficiency(),
    ]
).to_sdf()

test.show()

+-------------------+------------------+----------------------+
|primary_location_id|configuration_name|kling_gupta_efficiency|
+-------------------+------------------+----------------------+
|      usgs-01124000|        d_ensemble|             0.8706531|
|      usgs-01124000|             d_hbv|            0.85633445|
|      usgs-01124000|            d_prms|             0.8795865|
|      usgs-01124000|          d_sacsma|             0.8795865|
|      usgs-01153550|        d_ensemble|            0.87344384|
|      usgs-01153550|             d_hbv|             0.8016018|
|      usgs-01153550|            d_prms|            0.85472584|
|      usgs-01153550|          d_sacsma|            0.85472584|
|      usgs-01398500|        d_ensemble|            0.68338233|
|      usgs-01398500|             d_hbv|             0.6434177|
|      usgs-01398500|            d_prms|              0.726921|
|      usgs-01398500|          d_sacsma|              0.726921|
|      usgs-01447680|        d_ensemble|

In [12]:
ev.spark.stop()